In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

# scipy optimization
from scipy.optimize import nnls

### Reading in data

In [2]:
# create data DataFrame
logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])

In [3]:
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')

In [4]:
data.head()

,parcelid,logerror,month
0,17073783,0.0953,1
1,17088994,0.0198,3
2,17100444,0.0060,5
3,17102429,-0.0566,6
4,17109604,0.0573,8


### Reading in level-one data

In [86]:
l1data_linear_models = pd.read_csv("/home/anerdi/Desktop/Zillow/levelonedata/l1data_twostage_linear_models.csv.gz")
l1data_rfs = pd.read_csv("/home/anerdi/Desktop/Zillow/levelonedata/l1data_twostage_random_forests.csv.gz")

In [87]:
l1data = pd.merge(l1data_linear_models, l1data_rfs, on = 'parcelid')
l1data = pd.merge(l1data, data[['parcelid','logerror']], on = 'parcelid')

In [88]:
l1data.head()

,parcelid,stacked_rfs_ridge,stacked_rfs_enet,stacked_rfs_lasso,stacked_rfs_larm,stacked_rfs_huber,stacked_annrfs_ridge,stacked_annrfs_enet,stacked_annrfs_lasso,stacked_annrfs_larm,...,logistic_lasso,logistic_larm,logistic_huber,stacked_rfs_rf,stacked_rfs_rf_overfit,stacked_annrfs_rf,stacked_annrfs_rf_overfit,logistic_rf,logistic_rf_overfit,logerror
0,17073783,0.024176,0.025774,0.026020,0.027547,0.014576,0.018770,0.019923,0.020110,0.021019,...,0.019298,0.020122,0.010649,0.027348,0.020810,0.021123,0.014382,0.020268,0.013499,0.0953
1,17088994,0.004663,0.003873,0.003904,0.004764,0.000855,0.006679,0.005929,0.006016,0.007209,...,0.008818,0.010454,0.002048,0.004381,0.010182,0.006600,0.012431,0.009545,0.015414,0.0198
2,17100444,0.012671,0.011883,0.011940,0.012623,0.007172,0.016844,0.016133,0.016214,0.017172,...,0.004321,0.004514,0.002524,0.023879,0.015154,0.028744,0.019880,0.015207,0.006728,0.0060
3,17102429,0.012099,0.010563,0.010574,0.010785,0.008539,0.020774,0.018797,0.018773,0.019321,...,0.010865,0.011089,0.008784,0.013072,0.013386,0.021420,0.021441,0.013370,0.013672,-0.0566
4,17109604,0.018363,0.017627,0.017683,0.015224,0.014375,0.016069,0.015316,0.015377,0.012670,...,0.020294,0.018116,0.017030,0.017354,0.012509,0.015158,0.010770,0.019841,0.014478,0.0573


### Finding optimal weights using nnls

In [89]:
l1data.columns

Index(['parcelid', 'stacked_rfs_ridge', 'stacked_rfs_enet',
       'stacked_rfs_lasso', 'stacked_rfs_larm', 'stacked_rfs_huber',
       'stacked_annrfs_ridge', 'stacked_annrfs_enet', 'stacked_annrfs_lasso',
       'stacked_annrfs_larm', 'stacked_annrfs_huber', 'logistic_ridge',
       'logistic_enet', 'logistic_lasso', 'logistic_larm', 'logistic_huber',
       'stacked_rfs_rf', 'stacked_rfs_rf_overfit', 'stacked_annrfs_rf',
       'stacked_annrfs_rf_overfit', 'logistic_rf', 'logistic_rf_overfit',
       'logerror'],
      dtype='object')

In [ ]:
models = ['']

In [19]:
# Set up quadratic objective function
# V = l1_data.iloc[:,1:].values
X = l1data.iloc[:,1:-1].values
y = l1data.logerror.values

In [14]:
alpha, norm = nnls(A=V,b=y)

In [15]:
norm

48.150167385871036

In [16]:
alpha

array([ 1.33773319,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.5384619 ])

### Finding optimal weights using Ridge Regression

In [92]:
X = l1data[['stacked_rfs_huber',
             'stacked_rfs_lasso',
             'stacked_annrfs_rf',
             'logistic_lasso',
             'stacked_annrfs_huber',
             'logistic_rf',
             'logistic_huber',
             'stacked_rfs_enet',
             'stacked_rfs_larm',
             'stacked_annrfs_enet',
             'logistic_ridge',
             'stacked_annrfs_larm',
             'stacked_annrfs_lasso',
             'stacked_rfs_ridge',
             'logistic_enet',
             'stacked_annrfs_ridge',
             'logistic_larm']].values

y = l1data.logerror.values

In [94]:
from sklearn.neural_network import MLPRegressor

In [102]:
stack = MLPRegressor(hidden_layer_sizes=(100,100),random_state=19, early_stopping=True)

In [103]:
stack.fit(X,y)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=19, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [104]:
stack.score(X,y)

0.038529007695594752

In [105]:
(abs(stack.predict(X) - y)).mean()

0.066797194441360772

### Pickle the NN

In [106]:
from sklearn.externals import joblib

In [107]:
joblib.dump(stack, '/home/anerdi/Desktop/Zillow/submissions/stage2_stacked_NN.pkl') 

['/home/anerdi/Desktop/Zillow/submissions/stage2_stacked_NN.pkl']